# <center>Домашнее задание по занятию 21 "finetuning": Катастрофическое забывание (необязательное)

## Цель: Проверить влияние fine-tuning на исходную модель
1. Дообучить готовую модель на imagenette2 и проверить качество
2. Сохранить последний слой обученной модели и заменить его на новый для CIFAR10
2. Дообучить модель решать датасет CIFAR10 и проверить качество
3. Вернуть оригинальный последний слой модели и проверить качество на imagenette2
4. Дообучить только последний слой на imagenette2 и проверить удалось ли добиться исходного качества.

In [1]:
import torch
from torch import nn
from torchvision import models, datasets, transforms
import time
from tqdm.auto import tqdm
import os

In [2]:
def set_requires_grad(model, value=False):
    for param in model.parameters():
        param.requires_grad = value

input_size = 224
batch_size = 4
normalize = transforms.Compose([
    transforms.Resize(input_size),
    transforms.CenterCrop(input_size),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

device = torch.device("cuda:0")
num_classes = 10

criterion = nn.CrossEntropyLoss()

def train_model(model, dataloaders, criterion, optimizer, num_epochs=3):
    since = time.time()

    acc_history = {'train': [], 'val': []}
    loss_history = {'train': [], 'val': []}

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            n_batches = len(dataloaders[phase])
            for inputs, labels in tqdm(dataloaders[phase], total=n_batches):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            loss_history[phase].append(epoch_loss)
            acc_history[phase].append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

    return model, acc_history

def test_model(model, testloader, criterion):
    n_batches = len(testloader)
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in tqdm(testloader, total=n_batches):
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        _, preds = torch.max(outputs, 1)

        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / len(testloader.dataset)
    acc = running_corrects.double() / len(testloader.dataset)
    print('Val Loss: {:.4f} Val Acc: {:.4f}'.format(loss, acc))
    return acc, loss

In [19]:
# Load model

resnet18_model = models.resnet18(pretrained=True)
num_ftrs = resnet18_model.fc.in_features
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)

In [4]:
# Load dataset

# download dataset and unpack:
dataset_path = "imagenette2"
if not os.path.exists(dataset_path):
    !wget https://s3.amazonaws.com/fast-ai-imageclas/imagenette2.tgz
    !tar xvzf imagenette2.tgz

trainset_imagenette = datasets.ImageFolder(root='./%s/train/' % dataset_path, transform=normalize)
trainloader_imagenette = torch.utils.data.DataLoader(trainset_imagenette, batch_size=batch_size, shuffle=True, num_workers=2)

testset_imagenette = datasets.ImageFolder(root='./%s/val/' % dataset_path, transform=normalize)
testloader_imagenette = torch.utils.data.DataLoader(testset_imagenette, batch_size=batch_size, shuffle=False, num_workers=2)

loaders_imagenette = {'train': trainloader_imagenette, 'val': testloader_imagenette}

Streaming output truncated to the last 5000 lines.
imagenette2/train/n02102040/n02102040_1343.JPEG
imagenette2/train/n02102040/n02102040_487.JPEG
imagenette2/train/n02102040/n02102040_2516.JPEG
imagenette2/train/n02102040/n02102040_503.JPEG
imagenette2/train/n02102040/n02102040_2634.JPEG
imagenette2/train/n02102040/n02102040_5028.JPEG
imagenette2/train/n02102040/n02102040_5214.JPEG
imagenette2/train/n02102040/n02102040_3806.JPEG
imagenette2/train/n02102040/n02102040_467.JPEG
imagenette2/train/n02102040/n02102040_8208.JPEG
imagenette2/train/n02102040/n02102040_7124.JPEG
imagenette2/train/n02102040/n02102040_213.JPEG
imagenette2/train/n02102040/n02102040_35.JPEG
imagenette2/train/n02102040/n02102040_3997.JPEG
imagenette2/train/n02102040/n02102040_7617.JPEG
imagenette2/train/n02102040/n02102040_3699.JPEG
imagenette2/train/n02102040/n02102040_5136.JPEG
imagenette2/train/n02102040/n02102040_3593.JPEG
imagenette2/train/n02102040/n02102040_1148.JPEG
imagenette2/train/n02102040/n02102040_5376.

## 1. Дообучить готовую модель на imagenette2 и проверить качество

In [20]:
train_optimizer = torch.optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)
resnet18_model = resnet18_model.to(device)

In [21]:
# train model
train_model(resnet18_model, loaders_imagenette, criterion, train_optimizer, num_epochs=5)

Epoch 0/4
----------



train Loss: 0.5221 Acc: 0.8435



val Loss: 0.2961 Acc: 0.9131

Epoch 1/4
----------



train Loss: 0.3482 Acc: 0.8949



val Loss: 0.2184 Acc: 0.9371

Epoch 2/4
----------


...


train Loss: 0.2004 Acc: 0.9403



val Loss: 0.1742 Acc: 0.9536

Epoch 4/4
----------



train Loss: 0.1525 Acc: 0.9553



val Loss: 0.1704 Acc: 0.9536

Training complete in 15m 35s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

## 2. Сохранить последний слой обученной модели и заменить его на новый для CIFAR10

### сохраняем слой для imagenette

In [22]:
# save fc layer trained on imagenette
imagenette_fc = resnet18_model.fc
# save model
torch.save(resnet18_model, "./resnet18_model_imagenette.pth")

In [8]:
trainset_cifar10 = datasets.CIFAR10(root='./data', train=True, download=True, transform=normalize)
trainloader_cifar10 = torch.utils.data.DataLoader(trainset_cifar10, batch_size=batch_size, shuffle=True, num_workers=2)

testset_cifar10 = datasets.CIFAR10(root='./data', train=False, download=True, transform=normalize)
testloader_cifar10 = torch.utils.data.DataLoader(testset_cifar10, batch_size=batch_size, shuffle=False, num_workers=2)

loaders_cifar10 = {'train': trainloader_cifar10, 'val': testloader_cifar10}

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


### заменяем на слой для CIFAR10

In [9]:
num_ftrs = resnet18_model.fc.in_features
set_requires_grad(resnet18_model, False)
resnet18_model.fc = nn.Linear(num_ftrs, num_classes)
resnet18_model = resnet18_model.to(device)

## 3. Дообучить модель решать датасет CIFAR10 и проверить качество

In [11]:
pretrain_optimizer = torch.optim.SGD(resnet18_model.fc.parameters(), lr=0.001, momentum=0.9)
train_model(resnet18_model, loaders_cifar10, criterion, pretrain_optimizer, num_epochs=2)
# Train
set_requires_grad(resnet18_model, True)
train_optimizer = torch.optim.SGD(resnet18_model.parameters(), lr=0.001, momentum=0.9)
train_model(resnet18_model, loaders_cifar10, criterion, train_optimizer, num_epochs=1)

Epoch 0/1
----------



train Loss: 1.4389 Acc: 0.5165



val Loss: 1.1278 Acc: 0.6376

Epoch 1/1
----------



train Loss: 1.3163 Acc: 0.5652



val Loss: 1.2141 Acc: 0.6197

Training complete in 10m 50s
Epoch 0/0
----------



train Loss: 0.7444 Acc: 0.7656



val Loss: 0.3724 Acc: 0.8770

Training complete in 13m 28s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

### сохраняем слой для CIFAR10

In [12]:
torch.save(resnet18_model, "./resnet18_model_cifar10.pth")

## 4. Вернуть оригинальный последний слой модели и проверить качество на imagenette2

In [27]:
loaded_resnet18_model_imagenette = torch.load("./resnet18_model_imagenette.pth")
loaded_resnet18_model_cifar10 = torch.load("./resnet18_model_cifar10.pth")

In [28]:
loaded_resnet18_model_cifar10.fc = loaded_resnet18_model_imagenette.fc

#### с возвращенным оригинальным imagenette слоем. Проверяем качество на val imagenette

In [29]:
acc, loss = test_model(loaded_resnet18_model_cifar10, testloader_imagenette, criterion)


Val Loss: 1.3537 Val Acc: 0.5521


**Результат и вывод:**   
- исходный дообученный  val acc - **0.9536**
- после cifar10 с заменой последнего на оригинальный imagenette - **0.5521**
- качество ожидаемо **ухудшилось**

## 5. Дообучить только последний слой на imagenette2 и проверить удалось ли добиться исходного качества.

In [39]:
set_requires_grad(loaded_resnet18_model_cifar10, True)

In [40]:
loaded_resnet18_model_cifar10.fc = loaded_resnet18_model_imagenette.fc

In [41]:
loaded_resnet18_model_cifar10.to(device);

In [42]:
cifar10_imagenette_train_optimizer = torch.optim.SGD(loaded_resnet18_model_cifar10.fc.parameters(), lr=0.001, momentum=0.9)

train_model(loaded_resnet18_model_cifar10, loaders_imagenette, criterion, cifar10_imagenette_train_optimizer, num_epochs=5)

Epoch 0/4
----------



train Loss: 0.8709 Acc: 0.7277



val Loss: 0.5238 Acc: 0.8326

Epoch 1/4
----------



train Loss: 0.7141 Acc: 0.7771



val Loss: 0.4518 Acc: 0.8586

Epoch 2/4
----------


...


train Loss: 0.6542 Acc: 0.7961



val Loss: 0.4575 Acc: 0.8629

Epoch 4/4
----------



train Loss: 0.6008 Acc: 0.8124



val Loss: 0.4382 Acc: 0.8632

Training complete in 15m 0s


(ResNet(
   (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
   (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (relu): ReLU(inplace=True)
   (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
   (layer1): Sequential(
     (0): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU(inplace=True)
       (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     )
     (1): BasicBlock(
       (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
       (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
       (relu): ReLU

### Результат и вывод:
- исходного качества ожидаемо добиться не удалось: **0.8632 против 0.9536**